# 04 – Visualización de resultados TDA

Este notebook genera las figuras finales utilizadas en el análisis topológico de criptomonedas presentado en el **SEIO 2023 (Elche)**.

Incluye:

1. Carga de retornos logarítmicos
2. Carga de normas L¹ y L² (H₁) del pipeline TDA
3. Gráficos finales:
   - Retornos individuales
   - Normas L¹ y L² por ventana
   - Comparaciones entre ventanas
   - Suavizados por medias móviles

Los resultados se guardan en:
`../data/analysis/`.

In [ ]:
# =============================================
# 1. Librerías
# =============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.style.use("seaborn-v0_8-deep")

print("Librerías cargadas correctamente.")

## 2. Cargar datos procesados

In [ ]:
processed_dir = Path("../data/processed")
analysis_dir = Path("../data/analysis")
analysis_dir.mkdir(parents=True, exist_ok=True)

# Archivos
file_returns = processed_dir / "criptos_retornos_log_2018_2020.csv"
file_norms = processed_dir / "tda_landscapes_H1_norms.csv"

# Carga
log_returns = pd.read_csv(file_returns, index_col=0, parse_dates=True)
tda_norms = pd.read_csv(file_norms, index_col=0, parse_dates=True)

print("Retornos shape:", log_returns.shape)
print("Normas TDA shape:", tda_norms.shape)

## 3. Gráfico de retornos logarítmicos

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

for ax, col in zip(axes, log_returns.columns):
    ax.plot(log_returns.index, log_returns[col], linewidth=0.7)
    ax.axhline(0, color="black", linestyle="--", linewidth=0.7)
    ax.set_title(f"Retornos logarítmicos: {col}")
    ax.grid(True)

plt.tight_layout()
plt.savefig(analysis_dir / "retornos_logaritmicos.png", dpi=300, bbox_inches="tight")
plt.show()

## 4. Normas L¹ y L² (H₁) por ventana

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ax[0].plot(tda_norms.index, tda_norms["L1"], label="L¹(H₁)", linewidth=1)
ax[0].set_ylabel(r"$\|L\|_1$")
ax[0].grid(True)
ax[0].set_title("Norma L¹ del Persistence Landscape (H₁)")

ax[1].plot(tda_norms.index, tda_norms["L2"], label="L²(H₁)", linewidth=1, color="darkorange")
ax[1].set_ylabel(r"$\|L\|_2$")
ax[1].grid(True)
ax[1].set_title("Norma L² del Persistence Landscape (H₁)")

plt.xlabel("Fecha")
plt.tight_layout()
plt.savefig(analysis_dir / "normas_L1_L2_H1.png", dpi=300, bbox_inches="tight")
plt.show()

## 5. Comparación entre ventanas (si existen varias en el archivo)
Este apartado genera gráficos solo si el archivo contiene columnas como:
- `L1_w30`, `L1_w60`, `L1_w90`
- `L2_w30`, `L2_w60`, `L2_w90`

In [ ]:
columns_L1 = [c for c in tda_norms.columns if c.startswith("L1_w")]
columns_L2 = [c for c in tda_norms.columns if c.startswith("L2_w")]

if columns_L1:
    plt.figure(figsize=(14, 5))
    for col in columns_L1:
        plt.plot(tda_norms.index, tda_norms[col], linewidth=1, label=col)
    plt.grid(True)
    plt.title("Comparación normas L¹ por tamaños de ventana")
    plt.legend()
    plt.savefig(analysis_dir / "comparacion_L1_ventanas.png", dpi=300, bbox_inches="tight")
    plt.show()

if columns_L2:
    plt.figure(figsize=(14, 5))
    for col in columns_L2:
        plt.plot(tda_norms.index, tda_norms[col], linewidth=1, label=col)
    plt.grid(True)
    plt.title("Comparación normas L² por tamaños de ventana")
    plt.legend()
    plt.savefig(analysis_dir / "comparacion_L2_ventanas.png", dpi=300, bbox_inches="tight")
    plt.show()

## 6. Suavizado con medias móviles

In [ ]:
tda_smoothed = tda_norms.rolling(window=20).mean()

plt.figure(figsize=(14,5))
plt.plot(tda_smoothed.index, tda_smoothed["L1"], label="L1 suavizada", linewidth=1.5)
plt.plot(tda_smoothed.index, tda_smoothed["L2"], label="L2 suavizada", linewidth=1.5)
plt.grid(True)
plt.title("Normas L¹ y L² suavizadas (media móvil 20 días)")
plt.legend()
plt.savefig(analysis_dir / "normas_suavizadas.png", dpi=300, bbox_inches="tight")
plt.show()